In [9]:
import pandas as pd
import requests
import json
import os

In [7]:
api_key = "d3e02eb9569daeddcf8d7fca46f987c7"

In [10]:
URL = 'https://v1.formula-1.api-sports.io/teams'

# Correct way: API key goes in headers
headers = {
    'X-RapidAPI-Key': api_key,
    'X-RapidAPI-Host': 'v1.formula-1.api-sports.io'
}

response = requests.get(URL, headers=headers)

# Check the response status
if response.status_code == 200:
    team_data = response.json()
    print(json.dumps(team_data, indent=4))
else:
    print(f"Error: {response.status_code} - {response.text}")

{
    "get": "teams",
    "parameters": [],
    "errors": [],
    "results": 18,
    "response": [
        {
            "id": 1,
            "name": "Red Bull Racing",
            "logo": "https://media.api-sports.io/formula-1/teams/1.png",
            "base": "Milton Keynes, United Kingdom",
            "first_team_entry": 1997,
            "world_championships": 6,
            "highest_race_finish": {
                "position": 1,
                "number": 128
            },
            "pole_positions": 110,
            "fastest_laps": 100,
            "president": "Dietrich Mateschitz",
            "director": "Laurent Mekies",
            "technical_manager": "Pierre Wach\u00e9",
            "chassis": "RB21",
            "engine": "Honda RBPT",
            "tyres": "Pirelli"
        },
        {
            "id": 2,
            "name": "McLaren Racing",
            "logo": "https://media.api-sports.io/formula-1/teams/2.png",
            "base": "Woking, United Kingdom",
       

In [13]:
data_team = team_data['response']
print(json.dumps(data_team,indent=6))

[
      {
            "id": 1,
            "name": "Red Bull Racing",
            "logo": "https://media.api-sports.io/formula-1/teams/1.png",
            "base": "Milton Keynes, United Kingdom",
            "first_team_entry": 1997,
            "world_championships": 6,
            "highest_race_finish": {
                  "position": 1,
                  "number": 128
            },
            "pole_positions": 110,
            "fastest_laps": 100,
            "president": "Dietrich Mateschitz",
            "director": "Laurent Mekies",
            "technical_manager": "Pierre Wach\u00e9",
            "chassis": "RB21",
            "engine": "Honda RBPT",
            "tyres": "Pirelli"
      },
      {
            "id": 2,
            "name": "McLaren Racing",
            "logo": "https://media.api-sports.io/formula-1/teams/2.png",
            "base": "Woking, United Kingdom",
            "first_team_entry": 1966,
            "world_championships": 10,
            "highest_race_fin

In [14]:
rows = []
column_names = ['id','name','location','first_entry','Highest_accolades', 'Fastest_lap_time','times_achieved', 'president','director', 'chassis','engine','tyres']

for team in data_team:
    id = team['id']
    name = team['name']
    location = team['base']
    first_entry = team['first_team_entry']
    Highest_accolades = team["world_championships"]
    fastest_lap_time = team["highest_race_finish"]["position"]
    times_achieved = team["highest_race_finish"]["number"]
    president = team["president"]
    director = team["director"]
    chassis = team["chassis"]
    engine = team["engine"]
    tyres = team["tyres"]

    tuple_teams = (id,name,location,first_entry,Highest_accolades,fastest_lap_time,times_achieved,president,director,chassis,engine,tyres)

    rows.append(tuple_teams)

print(json.dumps(rows, indent=6))



[
      [
            1,
            "Red Bull Racing",
            "Milton Keynes, United Kingdom",
            1997,
            6,
            1,
            128,
            "Dietrich Mateschitz",
            "Laurent Mekies",
            "RB21",
            "Honda RBPT",
            "Pirelli"
      ],
      [
            2,
            "McLaren Racing",
            "Woking, United Kingdom",
            1966,
            10,
            1,
            203,
            "Zak Brown",
            "Andrea Stella",
            "MCL39",
            "Mercedes",
            "Pirelli"
      ],
      [
            3,
            "Scuderia Ferrari",
            "Maranello, Italy",
            1950,
            16,
            1,
            249,
            "John Elkann",
            "Fr\u00e9d\u00e9ric Vasseur",
            "SF-25",
            "Ferrari",
            "Pirelli"
      ],
      [
            4,
            "Force India",
            null,
            null,
            null,
    

In [15]:
df = pd.DataFrame(rows,columns= column_names)
df.head()

,id,name,location,first_entry,Highest_accolades,Fastest_lap_time,times_achieved,president,director,chassis,engine,tyres
0,1,Red Bull Racing,"Milton Keynes, United Kingdom",1997.0,6.0,1.0,128.0,Dietrich Mateschitz,Laurent Mekies,RB21,Honda RBPT,Pirelli
1,2,McLaren Racing,"Woking, United Kingdom",1966.0,10.0,1.0,203.0,Zak Brown,Andrea Stella,MCL39,Mercedes,Pirelli
2,3,Scuderia Ferrari,"Maranello, Italy",1950.0,16.0,1.0,249.0,John Elkann,Frédéric Vasseur,SF-25,Ferrari,Pirelli
3,4,Force India,None,NaN,NaN,NaN,NaN,"Subrata Roy, Vijay Mallya",Robert Fernley,None,"Ferrari V8, Mercedes V8, Mercedes V6 turbo","Bridgestone, Pirelli"
4,5,Mercedes-AMG Petronas,"Brackley, United Kingdom",1970.0,8.0,1.0,122.0,Markus Schäfer,Toto Wolff,W16,Mercedes,Pirelli


In [17]:
import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(df, preserve_index = False)

pq.write_table(
    table,
    "f1_output_parquet",
    row_group_size=100
)

In [22]:
table = pq.read_table("f1_output_parquet")